In [24]:
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from pathlib import Path
import joblib


In [25]:
gamesDF = pd.read_csv('./datasets/training_dataset_4szn_wp.csv')

In [26]:
gamesDF.head(5)

,GAME_ID,SEASON,PERIOD,SECONDS_REMAINING,HOME_SCORE,AWAY_SCORE,POINT_DIFF,HOME_WIN,HOME_WINS,HOME_LOSSES,AWAY_WINS,AWAY_LOSSES,HOME_L10_WINS,HOME_L10_LOSSES,AWAY_L10_WINS,AWAY_L10_LOSSES
0,22100001,2021-22,1,2880,0,0,0,1,0,0,0,0,0,0,0,0
1,22100001,2021-22,1,2875,0,0,0,1,0,0,0,0,0,0,0,0
2,22100001,2021-22,1,2862,0,0,0,1,0,0,0,0,0,0,0,0
3,22100001,2021-22,1,2859,0,0,0,1,0,0,0,0,0,0,0,0
4,22100001,2021-22,1,2847,0,0,0,1,0,0,0,0,0,0,0,0


In [27]:
gamesDF = gamesDF.drop(columns=['PERIOD', 'POINT_DIFF', 'HOME_L10_LOSSES', 'AWAY_L10_LOSSES'])

In [ ]:
X = gamesDF[['SECONDS_REMAINING','HOME_SCOREt','AWAY_SCORE', 'HOME_WINS', 'HOME_LOSSES', 'AWAY_WINS', 'AWAY_LOSSES', 'HOME_L10_WINS', 'AWAY_L10_WINS']] 
y = gamesDF['HOME_WIN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [29]:
model = LogisticRegression(max_iter=3000)
model.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [30]:
def run_model(model, show_confusion_matrix=False):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = model.score(X_test, y_test)

    if show_confusion_matrix:
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Win', 'Win'], yticklabels=['No Win', 'Win'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.show()

    return accuracy

def custom_predictions(model, custom_data):
    custom_df = pd.DataFrame(custom_data)
    custom_df = custom_df[['PERIOD','SECONDS_REMAINING','HOME_SCORE','AWAY_SCORE','POINT_DIFF', 'HOME_WINS', 'HOME_LOSSES', 'AWAY_WINS', 'AWAY_LOSSES', 'HOME_L10_WINS', 'AWAY_L10_WINS']]
    proba = model.predict_proba(custom_df)
    for i, p in enumerate(proba):
        print(f"Home WP for game {i+1}: {p[1]:.2%}")

def save_model(model, model_path_name):
    model_path = Path(model_path_name)
    joblib.dump(model, model_path)
    print(f"Model saved to {model_path.absolute()}")
    

In [31]:
custom_data = [
    {
        "PERIOD": 1,
        "SECONDS_REMAINING": 2288,
        "HOME_SCORE": 19,
        "AWAY_SCORE": 19,
        "POINT_DIFF": 0,
        "HOME_WINS": 19,
        "HOME_LOSSES": 24,
        "AWAY_WINS": 23,
        "AWAY_LOSSES": 18,
        "HOME_L10_WINS": 5,
        "AWAY_L10_WINS": 5    }
]
custom_predictions(model, custom_data)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- PERIOD
- POINT_DIFF


In [ ]:
accuracy = run_model(model)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 75.65%


In [ ]:
save_model(model, "lr_model_l10_s4.joblib")

Model saved to /Users/lemons/Documents/universidad/cs/pj09-sports-betting/ml/lr_model_l10_s4.joblib
